# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f5654ffad00>
├── 'a' --> tensor([[ 0.2550, -0.7420,  0.4048],
│                   [-0.5538,  0.2563, -0.6116]])
└── 'x' --> <FastTreeValue 0x7f5654ffae80>
    └── 'c' --> tensor([[ 1.1634,  0.9361, -0.2149,  1.2942],
                        [ 0.1373, -1.0982, -1.3636,  1.1513],
                        [ 0.0222,  0.3735,  0.1997,  1.9237]])

In [4]:
t.a

tensor([[ 0.2550, -0.7420,  0.4048],
        [-0.5538,  0.2563, -0.6116]])

In [5]:
%timeit t.a

70.4 ns ± 0.0531 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f5654ffad00>
├── 'a' --> tensor([[ 0.7204,  0.4966, -0.6914],
│                   [-1.1293, -0.8326, -0.3716]])
└── 'x' --> <FastTreeValue 0x7f5654ffae80>
    └── 'c' --> tensor([[ 1.1634,  0.9361, -0.2149,  1.2942],
                        [ 0.1373, -1.0982, -1.3636,  1.1513],
                        [ 0.0222,  0.3735,  0.1997,  1.9237]])

In [7]:
%timeit t.a = new_value

73.9 ns ± 0.108 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.2550, -0.7420,  0.4048],
               [-0.5538,  0.2563, -0.6116]]),
    x: Batch(
           c: tensor([[ 1.1634,  0.9361, -0.2149,  1.2942],
                      [ 0.1373, -1.0982, -1.3636,  1.1513],
                      [ 0.0222,  0.3735,  0.1997,  1.9237]]),
       ),
)

In [10]:
b.a

tensor([[ 0.2550, -0.7420,  0.4048],
        [-0.5538,  0.2563, -0.6116]])

In [11]:
%timeit b.a

64.7 ns ± 0.187 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.0684,  0.3273,  2.0427],
               [ 0.6930, -1.3571, -1.0194]]),
    x: Batch(
           c: tensor([[ 1.1634,  0.9361, -0.2149,  1.2942],
                      [ 0.1373, -1.0982, -1.3636,  1.1513],
                      [ 0.0222,  0.3735,  0.1997,  1.9237]]),
       ),
)

In [13]:
%timeit b.a = new_value

554 ns ± 7.52 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

868 ns ± 0.349 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.3 µs ± 17.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

146 µs ± 752 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

147 µs ± 860 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f5655012dc0>
├── 'a' --> tensor([[[ 0.2550, -0.7420,  0.4048],
│                    [-0.5538,  0.2563, -0.6116]],
│           
│                   [[ 0.2550, -0.7420,  0.4048],
│                    [-0.5538,  0.2563, -0.6116]],
│           
│                   [[ 0.2550, -0.7420,  0.4048],
│                    [-0.5538,  0.2563, -0.6116]],
│           
│                   [[ 0.2550, -0.7420,  0.4048],
│                    [-0.5538,  0.2563, -0.6116]],
│           
│                   [[ 0.2550, -0.7420,  0.4048],
│                    [-0.5538,  0.2563, -0.6116]],
│           
│                   [[ 0.2550, -0.7420,  0.4048],
│                    [-0.5538,  0.2563, -0.6116]],
│           
│                   [[ 0.2550, -0.7420,  0.4048],
│                    [-0.5538,  0.2563, -0.6116]],
│           
│                   [[ 0.2550, -0.7420,  0.4048],
│                    [-0.5538,  0.2563, -0.6116]]])
└── 'x' --> <FastTreeValue 0x7f5655012b80>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

33.6 µs ± 66.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f557a89ec10>
├── 'a' --> tensor([[ 0.2550, -0.7420,  0.4048],
│                   [-0.5538,  0.2563, -0.6116],
│                   [ 0.2550, -0.7420,  0.4048],
│                   [-0.5538,  0.2563, -0.6116],
│                   [ 0.2550, -0.7420,  0.4048],
│                   [-0.5538,  0.2563, -0.6116],
│                   [ 0.2550, -0.7420,  0.4048],
│                   [-0.5538,  0.2563, -0.6116],
│                   [ 0.2550, -0.7420,  0.4048],
│                   [-0.5538,  0.2563, -0.6116],
│                   [ 0.2550, -0.7420,  0.4048],
│                   [-0.5538,  0.2563, -0.6116],
│                   [ 0.2550, -0.7420,  0.4048],
│                   [-0.5538,  0.2563, -0.6116],
│                   [ 0.2550, -0.7420,  0.4048],
│                   [-0.5538,  0.2563, -0.6116]])
└── 'x' --> <FastTreeValue 0x7f557a89e970>
    └── 'c' --> tensor([[ 1.1634,  0.9361, -0.2149,  1.2942],
                        [ 0.1373, -1.0982, -1.3636,  1.1513],
                 

In [23]:
%timeit t_cat(trees)

31.3 µs ± 45.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

67.9 µs ± 82.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 1.1634,  0.9361, -0.2149,  1.2942],
                       [ 0.1373, -1.0982, -1.3636,  1.1513],
                       [ 0.0222,  0.3735,  0.1997,  1.9237]],
              
                      [[ 1.1634,  0.9361, -0.2149,  1.2942],
                       [ 0.1373, -1.0982, -1.3636,  1.1513],
                       [ 0.0222,  0.3735,  0.1997,  1.9237]],
              
                      [[ 1.1634,  0.9361, -0.2149,  1.2942],
                       [ 0.1373, -1.0982, -1.3636,  1.1513],
                       [ 0.0222,  0.3735,  0.1997,  1.9237]],
              
                      [[ 1.1634,  0.9361, -0.2149,  1.2942],
                       [ 0.1373, -1.0982, -1.3636,  1.1513],
                       [ 0.0222,  0.3735,  0.1997,  1.9237]],
              
                      [[ 1.1634,  0.9361, -0.2149,  1.2942],
                       [ 0.1373, -1.0982, -1.3636,  1.1513],
                       [ 0.0222,  0.3735,  0.1997,  1.9237]],

In [26]:
%timeit Batch.stack(batches)

83.6 µs ± 117 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 1.1634,  0.9361, -0.2149,  1.2942],
                      [ 0.1373, -1.0982, -1.3636,  1.1513],
                      [ 0.0222,  0.3735,  0.1997,  1.9237],
                      [ 1.1634,  0.9361, -0.2149,  1.2942],
                      [ 0.1373, -1.0982, -1.3636,  1.1513],
                      [ 0.0222,  0.3735,  0.1997,  1.9237],
                      [ 1.1634,  0.9361, -0.2149,  1.2942],
                      [ 0.1373, -1.0982, -1.3636,  1.1513],
                      [ 0.0222,  0.3735,  0.1997,  1.9237],
                      [ 1.1634,  0.9361, -0.2149,  1.2942],
                      [ 0.1373, -1.0982, -1.3636,  1.1513],
                      [ 0.0222,  0.3735,  0.1997,  1.9237],
                      [ 1.1634,  0.9361, -0.2149,  1.2942],
                      [ 0.1373, -1.0982, -1.3636,  1.1513],
                      [ 0.0222,  0.3735,  0.1997,  1.9237],
                      [ 1.1634,  0.9361, -0.2149,  1.2942],
                   

In [28]:
%timeit Batch.cat(batches)

154 µs ± 675 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

355 µs ± 5.37 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
